In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("restaurant-review-average").master('local[*]').getOrCreate()

In [2]:
sc = spark.sparkContext  # 필요 시 RDD API를 위해 사용

In [3]:
sc.defaultParallelism  # 필요

2

In [4]:
lines = sc.textFile("file:///home/jovyan/work/learning_spark_data/restaurant_reviews.csv")
lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [5]:
# 총 건수
# 첫 줄, 실제 데이터 분할
# map으로 split

In [6]:
header = lines.first()
header

'id,item,cateogry,reviews,'

In [7]:
data = lines.filter(lambda row: row != header)
data.take(5)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,']

In [8]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2] #카테고리
    
    # reviews는 정수로 parse
    reviews = fields[3] # 리뷰수
    reviews = int(reviews)
    
    return category, reviews

In [9]:
category_reviews = data.map(parse)
category_reviews

PythonRDD[5] at RDD at PythonRDD.scala:53

In [10]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [11]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
category_review_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [12]:
#같은 key(카테고리)끼리 x와 y의 (리뷰 수 합계, 개수 합계)를 누적
#첫번째 중식 x, 두번째 중식 y , [0] 은 리뷰수, [1] 은 개수 -> 같은 키끼리 행을 바꿔가며 계속 누적해 나간다.
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

| 구분    | `map()`                               | `mapValues()`                    |
| ----- | ------------------------------------- | -------------------------------- |
| 작동 대상 | (key, value) 전체                       | **value만 변환**                    |
| 리턴 형식 | `(new_key, new_value)`                | **key는 그대로**, value만 변경          |
| 예시    | `rdd.map(lambda x: (x[0], x[1] + 1))` | `rdd.mapValues(lambda v: v + 1)` |
| 용도    | key와 value 모두 수정할 때                   | value만 수정하고 싶을 때                 |


# Persist 사용

In [13]:
categoryReviews = data.map(parse).persist()
categoryReviews

PythonRDD[12] at RDD at PythonRDD.scala:53

In [14]:
result = categoryReviews.take(10)
result

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [15]:
result2 = categoryReviews.mapValues(lambda x : (x, 1)).collect()
result2

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [16]:
# 카테고리별 값1 합계 계산 (첫 번째 연산)
result_sum = categoryReviews \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
result_sum

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

In [17]:
# (카테고리, (점수, 1)) 형태로 변환
rdd_with_count = categoryReviews.map(lambda x: (x[0], (x[1], 1)))
rdd_with_count.take(3)

[('중식', (125, 1)), ('중식', (235, 1)), ('분식', (32, 1))]

In [18]:
# reduceByKey로 (점수 총합, 개수 총합) 집계
rdd_sums = rdd_with_count.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
rdd_sums.take(3)

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [19]:
# 평균 계산
rdd_avg = rdd_sums.mapValues(lambda x: round(x[0] / x[1], 2))

# 결과 출력
rdd_avg.collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.67), ('아시안', 312.0), ('패스트푸드', 17.5)]

In [20]:
# 기존 저장 레벨 해제
categoryReviews.unpersist()

PythonRDD[12] at RDD at PythonRDD.scala:53

In [21]:
from pyspark import StorageLevel
categoryReviews.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[12] at RDD at PythonRDD.scala:53

In [22]:
# 구조 확인하기
categoryReviews.take(3)

[('중식', 125), ('중식', 235), ('분식', 32)]

# Narrow Transformations
1:1 변환 -> 하나의 열을 다룰 때 다른 데이터가 필요 없는 경우
filter(), map(), flatMap(), sample(), union()



## flatMap()

In [23]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

[[1, 2], [2, 3], [3, 4]]

In [24]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

[1, 2, 2, 3, 3, 4]

In [25]:
# 텍스트를 단어로 쪼개서 flatMap()

In [26]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [27]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[33] at readRDDFromFile at PythonRDD.scala:289

In [29]:
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))  #job 추가 x
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

# wide transformations

## 집합 Transformation

In [31]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])
num1.intersection(num2).collect()

[4, 5]

In [32]:
# 합집합 구하기 - union

In [33]:
num_union = num1.union(num2)
num_union.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [34]:
# 차집합 구하기 - subtract

In [35]:
num1.subtract(num2).collect()

[1, 2, 3]

## 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)

In [48]:
# withReplacement : True -> 중복 추출
num_union.sample(True, 0.3).collect()

[1, 3, 5, 6, 9]

In [47]:
# withReplacement : False -> 중복 X
num_union.sample(False, 0.7).collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 10]

In [51]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
num_union.sample(True, 0.5, seed=42).collect()

[4, 5, 5, 5, 7]

In [49]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[70] at readRDDFromFile at PythonRDD.scala:289

In [50]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[75] at RDD at PythonRDD.scala:53

In [52]:
res = foodsGroup.collect()

In [53]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
마 ['마라탕']
떡 ['떡볶이']


In [54]:
kv_rdd = sc.parallelize([('apple', 1), ('banana', 2), ('cherry', 3)])

In [55]:
fruitsGroup = kv_rdd.groupBy(lambda x: len(x[0]))
fruitsGroup

PythonRDD[82] at RDD at PythonRDD.scala:53

In [57]:
response = fruitsGroup.collect()

In [58]:
for (k, v) in response:
    print(k, list(v))

6 [('banana', 2), ('cherry', 3)]
5 [('apple', 1)]


# join

In [63]:
# 두 RDD 조인하기
rdd1 = sc.parallelize([("apple", 2), ("banana", 1)])
rdd2 = sc.parallelize([("apple", "fruit"), ("banana", "fruit"), ("carrot", "vegetable")])

# 두 RDD 조인
joined_rdd = rdd1.join(rdd2)
print(joined_rdd.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

print("Joined RDD:", joined_rdd.collect())  # [("apple", (2, "fruit")), ("banana", (1, "fruit"))]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]
Joined RDD: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [64]:
# 왼쪽 외부 조인 (leftOuterJoin)
left_joined = rdd1.leftOuterJoin(rdd2)
print(left_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

# 오른쪽 외부 조인 (rightOuterJoin)
right_joined = rdd1.rightOuterJoin(rdd2)
print(right_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]

# subtractByKey 연산
subtract_result = rdd1.subtractByKey(sc.parallelize([("apple", "any")]))
print(subtract_result.collect())
# 출력 결과: [('banana', 1)]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]
[('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]
[('banana', 1)]


In [66]:
spark.stop()